#1. INITIALIZATION

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
import math
import datetime as dt
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

Importing essential libraries for data manipulation, visualization, machine learning models, and time series analysis. Setting seeds ensures reproducible results

##Data Fetching

In [3]:
# Define ticker and date range
ticker = 'AAPL'
start_date = '2015-01-01'
end_date = dt.datetime.now().strftime('%Y-%m-%d')

# Fetch stock data
data = yf.download(ticker, start=start_date, end=end_date)

# Check for NaN values and handle them
if data.isnull().values.any():
    print("NaN values found in data. Filling with forward fill method.")
    data.fillna(method='ffill', inplace=True)
    data.fillna(method='bfill', inplace=True)

print(f"Downloaded {len(data)} rows of data")
data.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Downloaded 2591 rows of data


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2015-01-02,24.320433,24.789802,23.879981,24.778679,212818400
2015-01-05,23.635286,24.169166,23.448429,24.089084,257142000
2015-01-06,23.637512,23.897778,23.274918,23.699798,263188400
2015-01-07,23.968967,24.069069,23.735394,23.846619,160423600
2015-01-08,24.889908,24.947745,24.180292,24.298192,237458000


Downloading historical stock data using yfinance API. We perform basic data cleaning by filling any missing values using forward and backward fill methods.

##Linear Regression Model - Data Preprocessing

In [4]:
def preprocess_for_linear_regression(data):
    """Preprocess data for linear regression model"""
    # Create a copy to avoid modifying original data
    df = data.copy()

    # Calculate moving averages
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA50'] = df['Close'].rolling(window=50).mean()

    # Calculate price momentum indicators
    df['Price_Change'] = df['Close'].pct_change()
    df['Price_Change_5d'] = df['Close'].pct_change(periods=5)
    df['Price_Change_20d'] = df['Close'].pct_change(periods=20)

    # Volume indicators
    df['Volume_Change'] = df['Volume'].pct_change()
    df['Volume_MA5'] = df['Volume'].rolling(window=5).mean()

    # Volatility indicators
    df['Volatility'] = df['Close'].rolling(window=20).std()

    # Remove NaN values that result from rolling calculations
    df = df.dropna()

    # Create features and target
    features = ['MA5', 'MA20', 'MA50', 'Price_Change', 'Price_Change_5d',
              'Price_Change_20d', 'Volume_Change', 'Volume_MA5', 'Volatility']

    X = df[features]
    y = df['Close']

    return X, y, df

# Preprocess data for linear regression
features_lr, target_lr, processed_df = preprocess_for_linear_regression(data)

# Display the first few rows of features
print("Linear Regression Features:")
features_lr.head()

Linear Regression Features:


Price,MA5,MA20,MA50,Price_Change,Price_Change_5d,Price_Change_20d,Volume_Change,Volume_MA5,Volatility
Ticker,,,,,,,,,
Date,,,,,,,,,
2015-03-16,27.681019,28.521400,26.677913,0.011004,-0.017225,-0.016761,-0.307811,219087920.0,0.618391
2015-03-17,27.794023,28.512578,26.758935,0.016726,0.020320,-0.006180,0.422274,204821120.0,0.619250
2015-03-18,28.072290,28.509787,26.860048,0.011257,0.050966,-0.001942,0.279242,201886640.0,0.618264
2015-03-19,28.208520,28.499179,26.956783,-0.007551,0.024508,-0.007396,-0.298164,199844080.0,0.616892
2015-03-20,28.311697,28.458980,27.039743,-0.012549,0.018691,-0.027799,0.499582,213338320.0,0.614151


Creating technical indicators that will serve as features for the linear regression model. These include moving averages, price momentum, volume indicators, and volatility measures which are common in financial analysis.

##Linear Regression Model - Train and Evaluate

In [5]:
# Split the data into training and testing sets (keeping time order)
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(
    features_lr, target_lr, test_size=0.2, shuffle=False
)

# Train the linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train_lr, y_train_lr)

# Make predictions
lr_train_pred = lr_model.predict(X_train_lr)
lr_test_pred = lr_model.predict(X_test_lr)

# Calculate metrics
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = math.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    # MAPE calculation with handling for zero values
    mape = np.mean(np.abs((y_true - y_pred) / np.maximum(1e-10, np.abs(y_true)))) * 100

    return {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'MAPE': mape
    }

lr_metrics = calculate_metrics(y_test_lr, lr_test_pred)
print("Linear Regression Model Metrics:")
for metric, value in lr_metrics.items():
    print(f"{metric}: {value:.4f}")

Linear Regression Model Metrics:
MSE: 7.0491
RMSE: 2.6550
MAE: 1.8070
R2: 0.9893
MAPE: 0.9011
